In [53]:
import pandas as pd
import json

import numpy as np
import matplotlib.pyplot as plt

import pylab
import nltk
import operator 
from collections import Counter
import regex as re

json_tweet_btp='C:/Users/Robin/Dropbox/Travaux_&_Rapports_Stages/UbicomLab-GATech/MeToo/code/Data_tweets/clean_tweet.json'
json_btp_com='C:/Users/Robin/Dropbox/Travaux_&_Rapports_Stages/UbicomLab-GATech/MeToo/code/balancetonporc_com/strories_balancetonporc_com.json'
json_metoo='C:/Users/Robin/Dropbox/Travaux_&_Rapports_Stages/UbicomLab-GATech/MeToo/code/Data-metoo/clean_tweet_metoo.json'

df_story = pd.read_json(json_btp_com, orient="columns")
df_tweet = pd.read_json(json_tweet_btp, orient="columns")
df_metoo= pd.read_json(json_metoo, orient="columns")



In [63]:
df_tweet

,fullname,hashtag,id,likes,mention,replies,retweets,text,timestamp,url,url_image,user
0,Sandra Muller,'#balancetonporc',918810180879515648,533,,82,377,toi aussi raconte en donnant le nom et le...,2017-10-13 12:06:32,,,LettreAudio
10,MeToo,'#balancetonporc',918929005411295232,10,,1,3,Un jour en contrat dapprentissage du haut de...,2017-10-13 19:58:42,,,Look_out_post
100,L'inconnue ☕️ ✊,'#balancetonporc',919271335024619520,2,,1,2,Il récupérait nos dans nos dossiers scolaires...,2017-10-14 18:39:00,,,_A_C_P
10000,Malice 💪,'#balancetonporc',919544934541217792,2,,2,0,Mon oncle ma violée à lâge de ans,2017-10-15 12:46:11,,,MaliceEnColere
10001,Mélisa,'#balancetonporc',919544936562819072,8,,3,8,ans cabinet davocat lassocié jai un moyen im...,2017-10-15 12:46:11,,,lawbylka87
10002,Nicolas Drouet,'#balancetonporc',919544936831307776,1,,0,0,ce type nest pas seulement un porc Cest aus...,2017-10-15 12:46:11,https://twitter.com/raptordissident/status/919...,,DrouetNico84
10003,David Abiker,'#balancetonporc',919544936990691328,87,,18,28,De tous ces je suggère à un éditeur de faire ...,2017-10-15 12:46:11,,,DavidAbiker
10004,Stormbringer,'#balancetonporc',919544939003990016,0,,0,0,Regroupement des beaufs de France qui critique...,2017-10-15 12:46:12,,,Alastor_Storm
10005,Pierre-Jérôme Adjedj,'#balancetonporc',919544940845256704,2,'@laurentbouvet',1,0,AH ben supercool et ça tient de l’oxymore,2017-10-15 12:46:12,,,impensif
10006,Veronique Perrot H.C,'#balancetonporc',919544960218779648,0,,0,0,le jour où tu découvre que ton violeur est le...,2017-10-15 12:46:17,,,VeroniquePerrot


In [78]:
# create text file with one tweet
import os
newpath = r'C:\Users\Robin\TweetsBTP' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

for i in range(len(df_tweet['text'])):
    filename='TweetsBTP/tweet'+str(i)+"_"+str(df_tweet['id'].iloc[i])+'.txt'
    file = open(filename,'w',encoding='utf-8')
    #for j in range(len(doc_stories_clean[i])):
    text=df_tweet['text'].iloc[i]
    file.write(text)
    file.write('\n')
    file.close()

## 1) Look at the key words

In [56]:
### win some time and delete stop word
import stop_words
sw = stop_words.get_stop_words('fr')+["""’"""]+['…']+['a']+['les']+['ça']+['cest']+['jai']

### key word for PTSD, depression
PTSD= ["ptsd","diagnos","tspt","syndrome","syndrom","sindrom","traumatique","sspt","anxiete","trouble","stress",
       "stres", ]
depression=["depression","depresion","deprime","deprimee","depressif"]
PTSD_en=['PTSD','ptsd','stress','anxiety','depression','syndrom','trauma','post-trauma']
HS=PTSD+depression
### function to find the relevant tweet and the story
def find_word(df,word):
    L=[]
    count_search = Counter()
    for i in range(len(df['text'])):
        text=nltk.word_tokenize(str.lower(df['text'].iloc[i]))
        terms_only = [term for term in text if term not in sw]
        for j in range(len(word)):
            if word[j] in terms_only:
                #count_search.update(terms_only)
                L.append(i)
    L=set(L)
    #for k in range(len(L)):
        #print(df['text'].iloc[L[k]],"/n")
    print("There are ",len(L)," users concerned: ",L, "soit",len(L)/len(df['text'])*100,"%")
    return(L)
        
    
    
    


In [61]:
L=find_word(df_metoo,PTSD_en)
print ("There are ",len(L)," users concerned: ",L, "soit",len(L)/len(df_metoo['text'])*100,"%")
#for i in find_word(df_metoo,PTSD_en):
#  #
#

There are  290  users concerned:  {200193, 107523, 46596, 73733, 91140, 129030, 131587, 73737, 151121, 183890, 122894, 23567, 108559, 89105, 137232, 73751, 73752, 165880, 118815, 160802, 53284, 28709, 117797, 35373, 36909, 141881, 108092, 109631, 134719, 30785, 191551, 74820, 51782, 194119, 196169, 126542, 10319, 143951, 37457, 11858, 32851, 38994, 44627, 82516, 93779, 122965, 137809, 141914, 39515, 153684, 118877, 202848, 73829, 100965, 186983, 153706, 56942, 91250, 24691, 148594, 79989, 122486, 181877, 53881, 120444, 36989, 135807, 112769, 163459, 176485, 179336, 47242, 60554, 73868, 55437, 125067, 201359, 198800, 107153, 26774, 82070, 194713, 119450, 179360, 153764, 111269, 145574, 167079, 96937, 132266, 82091, 136362, 107693, 76974, 161961, 187383, 117941, 203957, 38589, 43710, 82110, 63168, 94911, 94914, 158910, 115909, 94919, 1740, 149198, 171217, 184018, 34004, 27350, 94935, 112854, 71897, 163545, 179419, 202455, 157918, 120034, 77542, 81638, 74476, 2797, 111853, 157421, 111345,